# Get News

In [ ]:
!pip install worldnewsapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 591.5 kB/s eta 0:00:00


In [ ]:
import worldnewsapi
import pandas as pd
from worldnewsapi.rest import ApiException

In [ ]:
newsapi_configuration = worldnewsapi.Configuration(api_key={'apiKey': <INSERT YOUR APIKEY>})

In [ ]:
# The text to match in the news content
text = 'european election 2024'
# Article's language
language = 'en'
# Date range
earliest_date = '2024-06-01'
latest_date = '2024-06-12'

try:
  newsapi_instance = worldnewsapi.NewsApi(worldnewsapi.ApiClient(newsapi_configuration))

  # Max number of news to fetch
  max_res = 10000
  offset = 0
  all_results = []
  no_news = False

  while len(all_results) < max_res and  (not no_news):
    response = newsapi_instance.search_news(
      text=text,
      language=language,
      earliest_publish_date=earliest_date,
      latest_publish_date=latest_date,
      sort="publish-time",
      sort_direction="desc",
      offset=offset,
      number=100)

    offset += len(response.news)

    no_news = (len(response.news) == 0)

    print(f'Retrieved {str(len(response.news))} articles')

    all_results.extend(response.news)

except worldnewsapi.ApiException as e:
	print(f'Exception when calling NewsApi->search_news: {e}\n')

data = []

for article in all_results:
    print(f'\nTitle: {str(article.title)}')
    print(f'Date: {str(article.publish_date)}')
    print(f'Country: {str(article.source_country)}')
    print(f'Author: {str(article.author)}')
    print(f'URL: {str(article.url)}')
    print(f'Text: {str(article.text[:80])}...')

    data.append({
        'Title': article.title,
        'Date': article.publish_date,
        'Country': article.source_country,
        'Author': article.author,
        'URL': article.url,
        'Text': article.text
    })

df = pd.DataFrame(data)

df.head()

Exception when calling NewsApi->search_news: (402)
Reason: Payment Required
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 17 Jul 2024 12:19:30 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Allow-Control-Allow-Origin': '*', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Authorization, UserId, Hash, Name, Password, Accept, User-Agent, Accept-Language, x-api-key, Content-Language, Content-Type, Cache-Control, Origin, X-Requested-With', 'Access-Control-Allow-Methods': 'GET, HEAD, POST, OPTIONS, DELETE, PUT', 'Allow-Control-Allow-Methods': 'GET, HEAD, POST, OPTIONS, DELETE, PUT', 'CF-Cache-Status': 'DYNAMIC', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=LT02zYmIrYsCGB62FC36x4jMDumIep44MhZ7shmx6dEmwYFyOv3RP8G7yD6%2BnC%2F%2B9MXYKh%2BsrkmekFOiC%2B%2F2f0Ps2JKCsBG2eo6jw3x%2FBADjGpLR%2Br13%2F%2FPBpzSS3drsSC9hvhlEuA%3D%3D"}],"group":"cf-nel","max_age":604800}', 'NEL': '{"su

""


In [ ]:
data_path = 'data.csv'

df.to_csv(data_path, index=False)

# News to MongoDB

In [ ]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.0 MB/s eta 0:00:00


In [ ]:
import csv
from pymongo import MongoClient

In [ ]:
client = MongoClient(<INSERT LINK TO MONGO CLIENT>)

db = client['WorldNews']
collection = db['News']

# Loading articles on MongoDB
with open(data_path, 'r') as f:
  reader = csv.DictReader(f)

  for row in reader:
    collection.insert_one(row)

# NER and RE

## REBEL + spacy

In [ ]:
!pip install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 29.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
import pandas as pd
import spacy
import transformers
import torch
import pymongo
import nltk
from nltk import tokenize
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

import spacy_component

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
mongo_uri = '<INSERT LINK TO MONGO>'

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

def get_data():
    mongo_client = get_mongo_client(mongo_uri)

    db = mongo_client["WorldNews"]
    collection = db["News"]

    cursor = collection.find()

    df = pd.DataFrame(list(cursor))
    df = df.drop(columns='_id')

    return df

In [ ]:
# Perform NER-RE on fetched data or all news in the database
latest_news = True

data = None
if latest_news:
    data = pd.read_csv(data_path)
else:
    data = get_data()
news = data['Text']
candidate_sentences = []

# For each article, split the article in sentences
for new in news:
  sentences = tokenize.sent_tokenize(str(new))
  candidate_sentences.extend(sentences)

print(f'{len(candidate_sentences)} sentences extracted from {len(news)} news')

Connection to MongoDB successful
72039 sentences extracted from 2000 news


In [ ]:
nlp = spacy.load("en_core_web_sm")

use_gpu = False
device = 0 if use_gpu else -1

nlp.add_pipe("rebel", after="senter", config={
    'device':device, # Number of the GPU, -1 if want to use CPU
    'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
    )

head_entities = []
tail_entities = []
relations = []

for i, input_sentence in enumerate(candidate_sentences):
    doc = nlp(input_sentence)
    doc_list = nlp.pipe([input_sentence])
    for value, rel_dict in doc._.rel.items():
        print(f"sentence {i}/{len(candidate_sentences)} | {value}: {rel_dict}")
        head_entities.append(rel_dict['head_span'])
        tail_entities.append(rel_dict['tail_span'])
        relations.append(rel_dict['relation'])

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

sentence 0/72039 | (11, 17): {'relation': 'member of political party', 'head_span': Peter Pellegrini, 'tail_span': Hlas}
sentence 0/72039 | (41, 48): {'relation': 'member of political party', 'head_span': Ľuboš Blaha, 'tail_span': Smer}
sentence 1/72039 | (50, 54): {'relation': 'present in work', 'head_span': Lord Voldemort, 'tail_span': Harry Potter series}
sentence 1/72039 | (54, 50): {'relation': 'characters', 'head_span': Harry Potter series, 'tail_span': Lord Voldemort}
sentence 2/72039 | (15, 9): {'relation': 'office contested', 'head_span': June 8 election, 'tail_span': MEP}
sentence 3/72039 | (5, 7): {'relation': 'officeholder', 'head_span': Russian President, 'tail_span': Vladimir Putin}
sentence 3/72039 | (7, 5): {'relation': 'position held', 'head_span': Vladimir Putin, 'tail_span': Russian President}
sentence 3/72039 | (26, 28): {'relation': 'chairperson', 'head_span': Smer, 'tail_span': Robert Fico}
sentence 3/72039 | (28, 26): {'relation': 'member of political party', 'he

KeyboardInterrupt: 

In [ ]:
# Converting relations and entities in str
head_entities = [str(entity) for entity in head_entities]
tail_entities = [str(entity) for entity in tail_entities]
realtions = [str(relation) for relation in relations]

In [ ]:
for i in range(len(head_entities)):
  print(f'{head_entities[i]} - {relations[i]} - {tail_entities[i]}')

Peter Pellegrini - member of political party - Hlas
Ľuboš Blaha - member of political party - Smer
Lord Voldemort - present in work - Harry Potter series
Harry Potter series - characters - Lord Voldemort
June 8 election - office contested - MEP
Russian President - officeholder - Vladimir Putin
Vladimir Putin - position held - Russian President
Smer - chairperson - Robert Fico
Robert Fico - member of political party - Smer
Tibor Gašpar - member of political party - Smer
Chtelnica - located in the administrative territorial entity - Trnava Region
Trnava Region - contains administrative territorial entity - Chtelnica
Tisinec - located in the administrative territorial entity - Prešov Region
Stropkov - located in the administrative territorial entity - Prešov Region
Prešov Region - contains administrative territorial entity - Tisinec
Prešov Region - contains administrative territorial entity - Stropkov


In [ ]:
triplets = [(head_entity, relation, tail_entity) for head_entity, relation, tail_entity in zip(head_entities, relations, tail_entities)]
# Remove duplicates
triplets = list(set(triplets))

In [ ]:
triplets_text = ''

# From triplets to triplets text (which will be llama's input)
for triplet in triplets:
  triplets_text += (f'[{triplet[0].upper()}, {triplet[1].upper()}, {triplet[2].upper()}]\n')

In [ ]:
print(triplets_text)

[RUSSIAN PRESIDENT, OFFICEHOLDER, VLADIMIR PUTIN]
[CHTELNICA, LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY, TRNAVA REGION]
[TISINEC, LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY, PREŠOV REGION]
[PREŠOV REGION, CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY, TISINEC]
[LORD VOLDEMORT, PRESENT IN WORK, HARRY POTTER SERIES]
[VLADIMIR PUTIN, POSITION HELD, RUSSIAN PRESIDENT]
[TRNAVA REGION, CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY, CHTELNICA]
[TIBOR GAŠPAR, MEMBER OF POLITICAL PARTY, SMER]
[SMER, CHAIRPERSON, ROBERT FICO]
[STROPKOV, LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY, PREŠOV REGION]
[ĽUBOŠ BLAHA, MEMBER OF POLITICAL PARTY, SMER]
[HARRY POTTER SERIES, CHARACTERS, LORD VOLDEMORT]
[PETER PELLEGRINI, MEMBER OF POLITICAL PARTY, HLAS]
[JUNE 8 ELECTION, OFFICE CONTESTED, MEP]
[ROBERT FICO, MEMBER OF POLITICAL PARTY, SMER]
[PREŠOV REGION, CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY, STROPKOV]



## Llama 3b

In [ ]:
access_token = '<INSERT YOUR TOKEN'
login(token = access_token)

model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids('<|eot_id|>')
]

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def generate_text(messages,temp=0.6):
    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    outputs = pipeline(
        prompt,
        max_new_tokens=350,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temp,
        top_p=0.9,
    )

    generated_text = outputs[0]["generated_text"][len(prompt):]
    return generated_text.strip()

def classify_entities(text_triplets):
    messages = [
    {"role": "system", "content": "Answer questions"},
    {"role": "user", "content": f"""For each triple [entity1,relation,enitity2] categorize each entity by adding the category in square brackets next to the name (for both the first and second entities) from a single one among the categories
    proposed below (each entity must necessarily be associated with one of them, if not possible ignore the triple):
    {entity_list}

    Example of the output you will have to rigorously provide me considering the parentheses as well:
    [entity1[category_x],relation,entity2[category_y]]
    [entity1[category_z],relation,entity2[category_x]]
    [entity1[category_j],relationship,entity2[category_k]]
    [entity2[category_z],relation,entity5[category_x]]


    Do not provide more details than I have requested
    These are the lines you need to adapt:
    {text_triplets}"""},
]
    generated_response = generate_text(messages,0.3)
    return generated_response

In [ ]:
entity_list = [
'PERSON',
'ORGANIZATION',
'PARTY',
'LOCATION',
'DATE OR TIME PERIOD',
'MEASURE OR QUANTITY',
'PRODUCT OR SERVICE',
'EVENT',
'REGULATION',
'ECONOMIC SECTOR',
'NATURAL RESOURCE',
'TECHNOLOGY',
'INCIDENT'
]

In [ ]:
llama_output = ''
step = 10

for i in range(0, len(triplets_text), step):
    text = '\n'.join(triplets_text.split('\n')[i: i+step])
    if text != '':
        response = classify_entities(text)
        print(response)
        llama_output += (response + '\n')

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here are the categorized triples:

[RUSSIAN PRESIDENT[ORGANIZATION], OFFICEHOLDER, VLADIMIR PUTIN[PERSON]]
[CHTELNICA[LOCATION], LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY, TRNAVA REGION[LOCATION]]
[TISINEC[LOCATION], LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY, PREŠOV REGION[LOCATION]]
[PREŠOV REGION[LOCATION], CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY, TISINEC[LOCATION]]
[LORD VOLDEMORT[PERSON], PRESENT IN WORK, HARRY POTTER SERIES[PRODUCT OR SERVICE]]
[VLADIMIR PUTIN[PERSON], POSITION HELD, RUSSIAN PRESIDENT[ORGANIZATION]]
[TRNAVA REGION[LOCATION], CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY, CHTELNICA[LOCATION]]
[TIBOR GAŠPAR[PERSON], MEMBER OF POLITICAL PARTY, SMER[ORGANIZATION]]
[SMER[ORGANIZATION], CHAIRPERSON, ROBERT FICO[PERSON]]
[STROPKOV[LOCATION], LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY, PREŠOV REGION[LOCATION]]
Here are the categorized entities:

[ĽUBOŠ BLAHA[PERSON], MEMBER OF POLITICAL PARTY, SMER[ORGANIZATION]]
[HARRY POTTER SERIES[PRODUCT OR SERVICE

In [ ]:
def process_llama_output(llama_output, triplets=[]):
    for line in llama_output.split('\n'):
        if line != '' and line[0] == '[' and line[-1] == ']':
            line = line[1:-1]
            rel = [el.strip() for el in line.split(',') if el.strip() != '']

            if len(rel) == 3 and rel[0][-1] == ']' and rel[2][-1] == ']':
                first_entity = rel[0].split('[')[0].upper()
                first_entity_category = rel[0].split('[')[1][:-1].upper()
                rel[0] = f'{first_entity}: {first_entity_category}'

                rel[1] = rel[1].upper()

                second_entity = rel[2].split('[')[0].upper()
                second_entity_category = rel[2].split('[')[1][:-1].upper()
                rel[2] = f'{second_entity}: {second_entity_category}'

                if (first_entity_category in entity_list) and (second_entity_category in entity_list) and ('[' not in rel[1]) and (']' not in rel[1]):
                    triplets.append(rel)

classified_triplets = []

process_llama_output(llama_output, triplets=classified_triplets)

In [ ]:
print(classified_triplets)

[['RUSSIAN PRESIDENT: ORGANIZATION', 'OFFICEHOLDER', 'VLADIMIR PUTIN: PERSON'], ['CHTELNICA: LOCATION', 'LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY', 'TRNAVA REGION: LOCATION'], ['TISINEC: LOCATION', 'LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY', 'PREŠOV REGION: LOCATION'], ['PREŠOV REGION: LOCATION', 'CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY', 'TISINEC: LOCATION'], ['LORD VOLDEMORT: PERSON', 'PRESENT IN WORK', 'HARRY POTTER SERIES: PRODUCT OR SERVICE'], ['VLADIMIR PUTIN: PERSON', 'POSITION HELD', 'RUSSIAN PRESIDENT: ORGANIZATION'], ['TRNAVA REGION: LOCATION', 'CONTAINS ADMINISTRATIVE TERRITORIAL ENTITY', 'CHTELNICA: LOCATION'], ['TIBOR GAŠPAR: PERSON', 'MEMBER OF POLITICAL PARTY', 'SMER: ORGANIZATION'], ['SMER: ORGANIZATION', 'CHAIRPERSON', 'ROBERT FICO: PERSON'], ['STROPKOV: LOCATION', 'LOCATED IN THE ADMINISTRATIVE TERRITORIAL ENTITY', 'PREŠOV REGION: LOCATION'], ['ĽUBOŠ BLAHA: PERSON', 'MEMBER OF POLITICAL PARTY', 'SMER: ORGANIZATION'], ['HARRY POTTER SERIES: PRODUCT O

# Neo4J

In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase

URI = 'neo4j+s:<INSERT YOUR LINK TO DB'
AUTH = ('neo4j', '<INSERT YOUR TOKEN>')

def insert_neo4j(name, category):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        check = driver.execute_query('''
            MATCH (e:Entity {name: $name})
            WITH COUNT(e) > 0 as node_exists
            RETURN node_exists
            ''',
            name=name,
            database='neo4j'
        )[0]

        if check[0].data()['node_exists'] == False:
            summary = driver.execute_query(
                'CREATE (:Entity {name: $name, category: $category})',
                name=name,
                category=category,
                database_='neo4j',
            ).summary

            print('Created {nodes_created} nodes in {time} ms.'.format(
                nodes_created=summary.counters.nodes_created,
                time=summary.result_available_after)
            )

def create_rel(e1, e2, rel):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        records, summary, keys = driver.execute_query('''
                MATCH (e1:Entity {name: $e1})
                MATCH (e2:Entity {name: $e2})
                MERGE (e1)-[r:'''+f'`{rel}`'+''']-(e2)
                ''', e1=e1, e2=e2, rel=rel,
                database_='neo4j',
                )

for triplet in classified_triplets:
    first_entity = triplet[0].split(':')[0].strip()
    first_entity_category = triplet[0].split(':')[1].strip()

    second_entity = triplet[2].split(':')[0].strip()
    second_entity_category = triplet[2].split(':')[1].strip()

    insert_neo4j(first_entity, first_entity_category)
    insert_neo4j(second_entity, second_entity_category)

    create_rel(first_entity, second_entity, triplet[1])

TRUMP PERSON AMERICA LOCATION LOVES
